## M5 competition Forecasting - Accuracy

¿Puedes estimar con la mayor precisión posible la predicción de las ventas unitarias de varios productos vendidos por Walmart en los Estados Unidos?
Los datos qu[](http://)e nos proporcionan pertenecen a tiendas en tres estados de los EE. UU. (California, Texas y Wisconsin) e incluyen nivel de artículos, departamento, categorías de productos y detalles de la tienda. Además, tiene variables explicativas como precio, promociones, día de la semana y eventos especiales. Juntos, este conjunto de datos robusto se puede utilizar para mejorar la precisión del pronóstico.

Este trabajo tiene como objetivo hacer predicciones para las ventas de 28 días en el futuro. O, más precisamente, 28 días en un conjunto de datos público y 28 días en un conjunto de datos privado. Cuando quede un mes antes de la fecha límite, se liberarán las etiquetas de los 28 días públicos (sales_train_evaluation.csv).
Es bastante interesante que haya dos competencias para los mismos datos, la diferencia está en la métrica y los valores para predecir:


* En una parte se realizarán pronósticos puntuales y la métrica es Error escalado medio ponderado de raíz cuadrada (WRMSSE);

* En otra parte se realizarán predicciones para cuantiles y la métrica es Pérdida de Pinball Escalada Ponderada (WSPL



*A empezar*

## Importación de librerías

In [ ]:
import numpy as np
import pandas as pd
import os
from scipy.signal import hilbert
from scipy.signal import hann
from scipy.signal import convolve
import copy
import matplotlib.pyplot as plt
%matplotlib inline
from tqdm import tqdm_notebook
from sklearn.preprocessing import StandardScaler
from sklearn.svm import NuSVR, SVR
from sklearn.metrics import mean_absolute_error, f1_score
pd.options.display.precision = 15
from collections import defaultdict
import lightgbm as lgb
import xgboost as xgb
import catboost as cat
import time
from collections import Counter
import datetime
from catboost import CatBoostRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold, KFold, RepeatedKFold, GroupKFold, GridSearchCV, train_test_split, TimeSeriesSplit, RepeatedStratifiedKFold
from sklearn import metrics
from sklearn.metrics import classification_report, confusion_matrix
from sklearn import linear_model
import gc
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
from bayes_opt import BayesianOptimization
# import eli5
import shap
from IPython.display import HTML
import json
import altair as alt
from category_encoders.ordinal import OrdinalEncoder
import networkx as nx
import matplotlib.pyplot as plt
%matplotlib inline
from typing import List

import os
import time
import datetime
import json
import gc
from numba import jit

import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm_notebook

import lightgbm as lgb
import xgboost as xgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn import metrics
from typing import Any
from itertools import product
pd.set_option('max_rows', 500)
import re
from tqdm import tqdm
from joblib import Parallel, delayed

## Cargamos los datos

In [ ]:
path = '/kaggle/input/m5-forecasting-accuracy'
train_sales = pd.read_csv(f'{path}/sales_train_validation.csv')
calendar = pd.read_csv(f'{path}/calendar.csv')
submission = pd.read_csv(f'{path}/sample_submission.csv')
sell_prices = pd.read_csv(f'{path}/sell_prices.csv')

In [ ]:
calendar

Calendar: para cada fecha obtenemos información sobre el día de la semana / semana / mes / año, eventos y banderas que muestran si las tiendas permitieron compras con cupones de alimentos SNAP. Tenemos los datos para todas las fechas.

In [ ]:
train_sales.head()

Train_sales:  características del artículo y las ventas de cada día.

In [ ]:
sell_prices.head()

sell_prices: precios de venta de todos los artículos en todas las tiendas por semanas

In [ ]:
submission.head()

Nuestro archivo debe contener el pronóstico para los próximos 56 días (28 públicos, 28 privados). Hacemos predicciones para cada artículo en cada tienda.

## Exploración de datos:

### 1.	Comprobamos uno de los artículos:

> #hay que hacer predicciones de venta para todos los artículos de cada tienda. Vamos a explorar uno de ellos, a ver la información disponible 'HOBBIES_1_001'

In [ ]:
train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_001']

Podemos ver que este artículo se vende en 10 tiendas en 3 estados. Este artículo pertenece a la categoría HOBBIES y se vende en el departamento HOBBIES_1

In [ ]:
sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_1_001']

También tenemos los datos sobre los precios de venta en las 10 tiendas.

Ventas a lo largo del tiempo en diferentes tiendas

In [ ]:
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 6:].values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 5]);
plt.title('HOBBIES_1_002 sales')
plt.legend();

Hagamos que sea más suave: hacemos la media para el rolling de 30 días

In [ ]:
plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 6:].rolling(30).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 5]);
plt.title('HOBBIES_1_002 sales, rolling mean 30 days')
plt.legend();

plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 6:].rolling(60).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 5]);
plt.title('HOBBIES_1_002 sales, rolling mean 60 days')
plt.legend();

plt.figure(figsize=(12, 4))
for i in range(10):
    plt.plot(train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 6:].rolling(90).mean().values,
             label=train_sales.loc[train_sales['item_id'] == 'HOBBIES_1_002'].iloc[i, 5]);
plt.title('HOBBIES_1_002 sales, rolling mean 90 days')
plt.legend();


Conclusiones:
* Hay estacionalidad
* Las ventas son bajas lo que puede provocar que sean más difíciles de predecir.


In [ ]:
item_prices = sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_2_001']
for s in item_prices['store_id'].unique():
    small_df = item_prices.loc[item_prices['store_id'] == s]
    plt.plot(small_df['wm_yr_wk'], small_df['sell_price'], label=s)
plt.legend()
plt.title('HOBBIES_2_001 sell prices');

Qué situación tan extraña. Suponemos que hubo algunas ventas con precios rebajados.

### 2.Estudiamos una única tienda ('CA_1'):

In [ ]:
train_sales.loc[train_sales['store_id'] == 'CA_1']

Tenemos información sobre las ventas de 3049 artículos diferentes, que pertenecen a diferentes categorías y departamentos

In [ ]:
sell_prices.loc[sell_prices['store_id'] == 'CA_1']

También tenemos los datos sobre los precios de venta de todos los artículos en esta tienda.

In [ ]:
ca_1_sales = train_sales.loc[train_sales['store_id'] == 'CA_1']
pd.crosstab(ca_1_sales['cat_id'], ca_1_sales['dept_id'])

Esta tienda tiene 3 categorías: alimentos, entretenimiento y productos del hogar, que tienen 2-3 departamentos.

In [ ]:
plt.figure(figsize=(12, 4))
for d in ca_1_sales['dept_id'].unique():
    store_sales = ca_1_sales.loc[ca_1_sales['dept_id'] == d]
    store_sales.iloc[:, 6:].sum().rolling(30).mean().plot(label=d)
plt.title('CA_1 sales by department, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

Food es la categoría que más se vende.

In [ ]:
item_prices = sell_prices.loc[sell_prices['item_id'] == 'HOBBIES_2_001']
for s in item_prices['store_id'].unique():
    small_df = item_prices.loc[item_prices['store_id'] == s]
    plt.plot(small_df['wm_yr_wk'], small_df['sell_price'], label=s)
plt.legend()
plt.title('HOBBIES_2_001 sell prices');

In [ ]:
ca_1_prices = sell_prices.loc[sell_prices['store_id'] == 'CA_1']
ca_1_prices['dept_id'] = ca_1_prices['item_id'].apply(lambda x: x[:-4])


In [ ]:
plt.figure(figsize=(12, 6))
for d in ca_1_prices['dept_id'].unique():
    small_df = ca_1_prices.loc[ca_1_prices['dept_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('CA_1 mean sell prices by dept');

Podemos ver que los precios en HOBBIES_1 aumentan con el tiempo, pero para otras categorías son bastante estables.

### 3.Vemos la información de un solo departamento (diferentes artículos) (HOBBIES_1):

In [ ]:
train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1']

In [ ]:
train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1', 'item_id'].nunique()


Podemos ver que hay 416 artículos únicos en este departamento que se venden en todas las tiendas.

In [ ]:
sell_prices.loc[sell_prices['item_id'].str.contains('HOBBIES_1')]

In [ ]:
hobbies_1_sales = train_sales.loc[train_sales['dept_id'] == 'HOBBIES_1']
plt.figure(figsize=(12, 6))
for d in hobbies_1_sales['store_id'].unique():
    store_sales = hobbies_1_sales.loc[hobbies_1_sales['store_id'] == d]
    store_sales.iloc[:, 6:].sum().rolling(30).mean().plot(label=d)
plt.title('HOBBIES_1 sales by stores, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

Las ventas aumentan con el tiempo. Podemos ver que las tiendas CA_1 y CA_3 tienen mayores ventas que otras tiendas.

In [ ]:
sell_prices.head()

In [ ]:
hobbies_1_prices = sell_prices.loc[sell_prices['item_id'].str.contains('HOBBIES_1')]
plt.figure(figsize=(12, 6))
for d in hobbies_1_prices['store_id'].unique():
    small_df = hobbies_1_prices.loc[hobbies_1_prices['store_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('HOBBIES_1 mean sell prices by store');

No solo las ventas crecen con el tiempo, los precios también crecen. Podemos ver que hubo varios puntos de tiempo cuando el precio aumentó.

### 4.Información en un solo estado (CA)

In [ ]:
train_sales.loc[train_sales['state_id'] == 'CA']

In [ ]:
for col in ['item_id', 'dept_id', 'store_id']:
    print(f"{col} has {train_sales.loc[train_sales['state_id'] == 'CA', col].nunique()} unique values for CA state")

In [ ]:
ca_sales = train_sales.loc[train_sales['state_id'] == 'CA']
plt.figure(figsize=(12, 6))
for d in ca_sales['store_id'].unique():
    store_sales = ca_sales.loc[ca_sales['store_id'] == d]
    store_sales.iloc[:, 6:].sum().rolling(30).mean().plot(label=d)
plt.title('CA sales by stores, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

Podemos ver muchas cosas interesantes:

* La tienda ca_3 siempre tiene mayores ventas;
* ca_1 tiene una pequeña tendencia creciente;
* ca_2 tuvo un largo descenso y luego tuvo un fuerte aumento en las ventas;


In [ ]:
ca_prices = sell_prices.loc[sell_prices['store_id'].str.contains('CA')]
plt.figure(figsize=(12, 6))
for d in ca_prices['store_id'].unique():
    small_df = ca_prices.loc[ca_prices['store_id'] == d]
    grouped = small_df.groupby(['wm_yr_wk'])['sell_price'].mean()
    plt.plot(grouped.index, grouped.values, label=d)
plt.legend(loc=(1.0, 0.5))
plt.title('Mean sell prices by store in CA');

En varios puntos del tiempo los precios aumentaron

### 5.	Agregaciones por departamentos 

In [ ]:
train_sales.head()

In [ ]:
plt.figure(figsize=(12, 8))
dept_grouped_sales = train_sales.groupby(['dept_id']).sum()
for i, row in dept_grouped_sales.iterrows():
    plt.plot(row.values, label=i);
plt.legend(loc=(1.0, 0.5))
plt.title('Sales by departments');

In [ ]:
plt.figure(figsize=(12, 4))
for i, row in dept_grouped_sales.iterrows():
    plt.plot(row.rolling(30).mean().values, label=i);
plt.title('Sales by department, rolling mean 30 days')
plt.legend(loc=(1.0, 0.5));

plt.figure(figsize=(12, 4))
for i, row in dept_grouped_sales.iterrows():
    plt.plot(row.rolling(60).mean().values, label=i);
plt.title('Sales by department, rolling mean 60 days')
plt.legend(loc=(1.0, 0.5));

plt.figure(figsize=(12, 4))
for i, row in dept_grouped_sales.iterrows():
    plt.plot(row.rolling(90).mean().values, label=i);
plt.title('Sales by department, rolling mean 90 days')
plt.legend(loc=(1.0, 0.5));


El departamento Food_3 tiene más ventas que el resto